In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os 
import re 
import sys
from distutils.dir_util import copy_tree

import subprocess
import json

import subprocess
import time

# cwd = os.getcwd()
# cwd = r'C:\Users\Laxman\Desktop\woodSDATool_Eusef'
cwd = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/woodSDA/woodSDATool_Eusef'

sys.path.append(os.path.join(cwd, *['Codes','DesignTool']))
sys.path.append(os.path.join(cwd, *['Codes','ModelingTool']))
sys.path.append(os.path.join(cwd, *['Codes', 'PostProcessing']))

####################### Modules required for Design Tool ##################
from FinalShearWallDesign_allFloors import FinalShearWallDesign
from StiffnessBasedDesign import RDADesignIterationClass

####################### Modules required for OpenSees Modeling ##################
# os.chdir(os.path.join(cwd, *['Codes','ModelingTool']))
from BuildingModelClass import BuildingModel
from utils import *

####################### Modules required for Post-Processing ##################
import ExtractMaxEDP as extractedps
import ExtractPushoverData as epd
import csv
import math

####################### Modules required for Loss Assessment ##################
# and import pelicun classes and methods
# from pelicun.base import set_options, convert_to_MultiIndex
# from pelicun.assessment import Assessment

# pd.set_option('display.max_colwidth', 100)
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install pelicun==3.1.b6

### setting up directories and folders to store the building models

In [3]:
# Utility function directory 
UtilDirectory = os.path.join(cwd, *['Codes','ModelingTool'])
# Base directory is the main directory that models, model inputs and utility directory stores
BaseDirectory = cwd
# Model directory is where you want to store your model
ModelDirectory = os.path.join(cwd, 'BuildingModels')
# DB directory is where you store Database.csv (for steel section)
DBDirecctory = UtilDirectory

# If there is no model directory, create one
if os.path.isdir(ModelDirectory) != True:
    os.chdir(BaseDirectory)
    os.mkdir('BuildingModels')

## create a directory to story results
resultDirectory = os.path.join(cwd, 'Results')
if os.path.isdir(resultDirectory) != True:
    os.chdir(cwd)
    os.mkdir('Results')
    
# Read in building name(s) 
# Make sure the building name is consistent with the input folders
os.chdir(os.path.join(cwd, 'Codes'))
with open('BuildingNames_woodSDATest.txt', 'r') as f:
    BuildingList = f.read() 
BuildingList = BuildingList.split('\n')
BuildingList

['s4_96x48']

## Implement design module

No need to post-process the outputs of design module. The modeling module is able to read in the required inputs from design module's output

In [5]:
start = time.time()
# caseID = 'woodSDATestModel'
caseID = BuildingList[0]
basedirectory = os.path.join(cwd, *['BuildingInfo', caseID])
numFloors = 4
# direction = ['X', 'X', 'Y', 'Y', 'Y']
# wall_line_name = ['sw1', 'sw2', 'swA', 'swB', 'swC']
# numWallsPerLine = [2,2,2,2,2]
 
direction = ['X', 'X', 'X', 
             'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']
wall_line_name = ['gridA', 'gridB', 'gridC',
                  'grid1', 'grid2','grid3','grid4','grid5','grid6', 'grid7']
numWallsPerLine = [4, 4, 4,
                  2, 2, 4, 2, 4, 2, 2]
counter = 0

rda = RDADesignIterationClass(caseID, basedirectory, direction,numWallsPerLine, counter, wall_line_name,
                              weight_factor=1, seismic_design_level='Extreme', mat_ext_int='HWS_GWB')

# rda.maindf
# rda.maindf.to_csv(os.path.join(resultDirectory, "FinalDesignOutput.csv"))
stop = time.time()
print( stop - start, 'seconds')

5.085832834243774 seconds


In [6]:
rda.maindf

Shear Wall Assembly  \
swDesign_gridA_wall1 0  Use 1/2in GWB with 5d cooler nails @ 7in o.c. ...   
                     1  Use 19/32in WSP on 1 side with 10d nails @ 6o....   
                     2  Use 3/8in WSP on 1 side with 6d nails @ 2o.c. ...   
                     3  Use 15/32in WSP on 1 side with 10d nails @ 2o....   
swDesign_gridA_wall2 0  Use 1/2in GWB with 5d cooler nails @ 7in o.c. ...   
...                                                                   ...   
swDesign_grid7_wall1 3  Use 15/32in WSP on 1 side with 10d nails @ 2o....   
swDesign_grid7_wall2 0  Use 1/2in GWB with 5d cooler nails @ 7in o.c. ...   
                     1  Use 7/16in WSP on 1 side with 8d nails @ 6o.c....   
                     2  Use 7/16in WSP on 1 side with 8d nails @ 3o.c....   
                     3  Use 15/32in WSP on 1 side with 10d nails @ 2o....   

                        Ga(k/in)  level  LRFD(klf)  Drift(in)  D/C Ratio  \
swDesign_gridA_wall1 0       5.2      4      0.160   1.183003   0.809222   
                     1      19.0      3      0.544   0.812688   0.714019   
                     2      32.0      2      0.816   0.901774   0.952026   
                     3      52.0      1      1.232   0.921743   0.894849   
swDesign_gridA_wall2 0       5.2      4      0.160   1.141859   0.809368   
...                          ...    ...        ...        ...        ...   
swDesign_grid7_wall1 3      52.0      1      1.232   1.270957   0.805445   
swDesign_grid7_wall2 0       5.2      4      0.160   1.178956   0.728373   
                     1      15.0      3      0.384   1.031290   0.910466   
                     2      28.0      2      0.720   1.143055   0.971163   
                     3      52.0      1      1.232   1.270957   0.805445   

                        OpenSees Tag  
swDesign_gridA_wall1 0            14  
                     1             8  
                     2             6  
                     3            11  
swDesign_gridA_wall2 0            14  
...                              ...  
swDesign_grid7_wall1 3            11  
swDesign_grid7_wall2 0            14  
                     1             1  
                     2             2  
                     3            11  

[120 rows x 7 columns]